In [2]:
import pandas as pd
# import functions_barcelona as fb
# import datetime
# try:
#     from holidays_es import Province
#     festes=True
# except:
#     festes=False
    
# import numpy as np
# import os

# from sklearn.metrics import recall_score, precision_score, accuracy_score, roc_auc_score
# from sklearn.model_selection import KFold, cross_val_score, cross_validate, StratifiedKFold
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.naive_bayes import GaussianNB
# from sklearn.svm import LinearSVC
# from sklearn.preprocessing import MinMaxScaler, StandardScaler,Normalizer
# from xgboost import XGBClassifier

# # from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
# # from imblearn.under_sampling import RandomUnderSampler, NearMiss
# from sklearn.dummy import DummyClassifier
# import imblearn.under_sampling as under
# import imblearn.over_sampling as over
# import imblearn.combine as over_under
# from imblearn.pipeline import Pipeline, make_pipeline

In [28]:
accidents=pd.read_csv('./data/accidents_weather.csv')
accidents['street_name']=accidents.street_name.str.strip()
accidents.head()

,num_incident,district_code,district,neighborhood,street_name,weekday,year,month,day,hour,...,temp_avg,temp_max,temp_min,relative_humidity,precipitation,windspeed,wind_direction,max_windspeed,pressure,solar_radiation
0,2023S000001,3,Sants-Montjuïc,Sants,Badal / Càceres,Sunday,2023.0,January,1.0,10.0,...,11.9,12.2,11.4,94.0,0.0,7.2,227.0,11.5,1027.1,315.0
1,2023S000002,7,Horta-Guinardó,el Baix Guinardó,Encarnació,Sunday,2023.0,January,1.0,3.0,...,14.2,14.7,13.5,58.0,0.0,8.3,257.0,13.7,1025.4,0.0
2,2023S000003,1,Ciutat Vella,el Raval,Carme,Sunday,2023.0,January,1.0,3.0,...,14.2,14.7,13.5,58.0,0.0,8.3,257.0,13.7,1025.4,0.0
3,2023S000005,6,Gràcia,la Vila de Gràcia,Riera de Sant Miquel / Sèneca,Sunday,2023.0,January,1.0,17.0,...,12.8,13.0,12.6,53.0,0.0,4.3,229.0,10.4,1025.1,0.0
4,2023S000006,6,Gràcia,la Vila de Gràcia,Gran de Gràcia / Gràcia,Sunday,2023.0,January,1.0,17.0,...,12.8,13.0,12.6,53.0,0.0,4.3,229.0,10.4,1025.1,0.0


<a id='start'></a>
# Feature Engineering
 1. [**Classifying streets by level of accidents.**](#binning) We should be careful with this one. There is some leakege the way it is produced. Repeating the process once we split train/test would be recommended

<a id='binning'></a>
1. Classifying streets by level of accidents.
[Back](#start)

In [46]:
def binning_streets(df, last_year_train,num_bins=5):
    """binning the streets for train and adding it
    without leakage to test"""
    train=df[df.year<=last_year_train]
    df['street_count']=pd.qcut(
    train.street_name.map(
        train.street_name.value_counts().to_dict()),
                                  num_bins,
                                  labels=False)
    return df
accidents=binning_streets(accidents,2021)
accidents.head()

,num_incident,district_code,district,neighborhood,street_name,weekday,year,month,day,hour,...,temp_max,temp_min,relative_humidity,precipitation,windspeed,wind_direction,max_windspeed,pressure,solar_radiation,street_count
0,2023S000001,3,Sants-Montjuïc,Sants,Badal / Càceres,Sunday,2023.0,January,1.0,10.0,...,12.2,11.4,94.0,0.0,7.2,227.0,11.5,1027.1,315.0,NaN
1,2023S000002,7,Horta-Guinardó,el Baix Guinardó,Encarnació,Sunday,2023.0,January,1.0,3.0,...,14.7,13.5,58.0,0.0,8.3,257.0,13.7,1025.4,0.0,NaN
2,2023S000003,1,Ciutat Vella,el Raval,Carme,Sunday,2023.0,January,1.0,3.0,...,14.7,13.5,58.0,0.0,8.3,257.0,13.7,1025.4,0.0,NaN
3,2023S000005,6,Gràcia,la Vila de Gràcia,Riera de Sant Miquel / Sèneca,Sunday,2023.0,January,1.0,17.0,...,13.0,12.6,53.0,0.0,4.3,229.0,10.4,1025.1,0.0,NaN
4,2023S000006,6,Gràcia,la Vila de Gràcia,Gran de Gràcia / Gràcia,Sunday,2023.0,January,1.0,17.0,...,13.0,12.6,53.0,0.0,4.3,229.0,10.4,1025.1,0.0,NaN


In [45]:
def binning_streets(df, last_year_train,num_bins=5):
    """binning the streets for train and adding it
    without leakage to test"""
    train=df[df.year<=last_year_train]
    df['street_count']=pd.qcut(
    train.street_name.map(
        train.street_name.value_counts().to_dict()),
                                  num_bins,
                                  labels=False)
    return df
binning_streets(accidents,2021)

,num_incident,district_code,district,neighborhood,street_name,weekday,year,month,day,hour,...,temp_max,temp_min,relative_humidity,precipitation,windspeed,wind_direction,max_windspeed,pressure,solar_radiation,street_count
0,2023S000001,3,Sants-Montjuïc,Sants,Badal / Càceres,Sunday,2023.0,January,1.0,10.0,...,12.2,11.4,94.0,0.0,7.2,227.0,11.5,1027.1,315.0,NaN
1,2023S000002,7,Horta-Guinardó,el Baix Guinardó,Encarnació,Sunday,2023.0,January,1.0,3.0,...,14.7,13.5,58.0,0.0,8.3,257.0,13.7,1025.4,0.0,NaN
2,2023S000003,1,Ciutat Vella,el Raval,Carme,Sunday,2023.0,January,1.0,3.0,...,14.7,13.5,58.0,0.0,8.3,257.0,13.7,1025.4,0.0,NaN
3,2023S000005,6,Gràcia,la Vila de Gràcia,Riera de Sant Miquel / Sèneca,Sunday,2023.0,January,1.0,17.0,...,13.0,12.6,53.0,0.0,4.3,229.0,10.4,1025.1,0.0,NaN
4,2023S000006,6,Gràcia,la Vila de Gràcia,Gran de Gràcia / Gràcia,Sunday,2023.0,January,1.0,17.0,...,13.0,12.6,53.0,0.0,4.3,229.0,10.4,1025.1,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126309,2010S004496,9,Sant Andreu,la Trinitat Vella,Dalt (Besòs),Tuesday,2010.0,June,29.0,7.0,...,25.4,23.8,34.0,0.0,13.3,292.0,17.6,1018.3,463.0,4.0
126310,2010S004253,9,Sant Andreu,Sant Andreu,Llenguadoc,Friday,2010.0,June,18.0,8.0,...,19.5,19.0,56.0,0.0,6.8,229.0,18.0,1015.2,666.0,1.0
126311,2010S008126,9,Sant Andreu,Sant Andreu,Gran de Sant Andreu,Thursday,2010.0,November,25.0,23.0,...,5.3,4.9,63.0,0.0,17.6,347.0,34.9,1007.5,0.0,2.0
126312,2010S004114,9,Sant Andreu,el Bon Pastor,Caracas,Saturday,2010.0,June,12.0,20.0,...,17.0,16.8,91.0,0.0,11.9,39.0,15.8,1013.3,0.0,1.0


In [43]:
X_train=accidents[accidents.year>2021]
X_test=accidents[accidents.year<2022]
pd.qcut(
    accidents.street_name.map(
        X_train.street_name.value_counts().to_dict()),
                                  5,
                                  labels=False,
duplicates='drop')


0         0.0
1         0.0
2         0.0
3         0.0
4         1.0
         ... 
126309    4.0
126310    NaN
126311    1.0
126312    1.0
126313    NaN
Name: street_name, Length: 126314, dtype: float64